In [1]:
import csv
import numpy as np
import pandas as pd

In [7]:
x = []
y = []
with open ('/content/data.csv') as file:
  csv_reader = csv.reader(file)
  for row in csv_reader:
    if row[0]=='ID':
      continue
    datum = []
    datum.append(row[3])
    datum.append(row[4])
    if row[1]=='Female':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[2]))
    bmi = float(row[4])/(float(row[3])**2)
    datum.append(bmi)
    if row[5]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    if row[6]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[7]))
    datum.append(float(row[8]))
    if row[9]=='no':
      datum.append(0)
    elif row[9]=='Sometimes':
      datum.append(1)
    elif row[9]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    if row[5]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[11]))
    if row[12]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[13]))
    datum.append(float(row[14]))
    if row[15]=='no':
      datum.append(0)
    elif row[15]=='Sometimes':
      datum.append(1)
    elif row[15]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    if row[16]=='Walking':
      datum.append(0)
    elif row[16]=='Bike':
      datum.append(1)
    elif row[16]=='Public_Transportion':
      datum.append(2)
    elif row[16]=='Motorbike':
      datum.append(3)
    else:
      datum.append(4)
    if row[17]=='Insufficient_Weight':
      y.append(0)
    elif row[17]=='Normal_Weight':
      y.append(1)
    elif row[17]=='Overweight_Level_I':
      y.append(2)
    elif row[17]=='Overweight_Level_II':
      y.append(3)
    elif row[17]=='Obesity_Type_I':
      y.append(4)
    elif row[17]=='Obesity_Type_II':
      y.append(5)
    else:
      y.append(6)
    x.append(datum)
print(x)
print(y)
x = np.array(x)
y = np.array(y)

[['1.620109', '103.393354', 1, 20.156664, 39.39164836817944, 0, 0, 2.766441, 1.240424, 1, 0, 1.413218, 1, 0.165269, 0.862587, 0, 4], ['1.686033', '104.772164', 0, 25.289428, 36.85647299826425, 0, 0, 3.0, 3.0, 1, 0, 1.299194, 1, 0.234303, 0.946888, 1, 4], ['1.65', '73', 0, 37.0, 26.813590449954088, 0, 1, 3.0, 3.0, 1, 0, 2.0, 1, 1.0, 0.0, 1, 4], ['1.719509', '84.416515', 1, 34.204611, 28.55081516555847, 0, 0, 2.031246, 3.0, 1, 0, 2.803002, 1, 1.98448, 0.418623, 0, 4], ['1.849507', '121.560938', 1, 22.580038, 35.53710813782194, 0, 0, 3.0, 2.272801, 1, 0, 1.560064, 1, 0.79677, 0.0, 1, 4], ['1.59611', '42.848033', 0, 22.142432, 16.819196817270793, 1, 1, 3.0, 2.581015, 2, 1, 2.654702, 1, 0.902095, 0.0, 0, 4], ['1.7', '80.761409', 1, 24.108711, 27.94512422145329, 0, 0, 2.0, 3.0, 1, 0, 2.879402, 1, 0.0, 0.322405, 1, 4], ['1.555778', '42.3601', 0, 21.287999, 17.50095406313848, 1, 1, 2.318355, 1.082304, 2, 1, 1.220365, 1, 0.033328, 0.0, 0, 4], ['1.748524', '133.878843', 0, 21.028989, 43.78937584

In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 13.2 MB/s 
     |████████████████████████████████| 210 kB 73.6 MB/s 
     |████████████████████████████████| 81 kB 10.7 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 147 kB 71.3 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 112 kB 63.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=f6ce5e61c3473a206c674f98f8fd8a5ce922a1f99aaa8793cd95bb82e7f78af3
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


In [16]:
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from optuna.integration import LightGBMPruningCallback

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [59]:
def objective(trial, X, y):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "dart",
        "num_class": 7,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.5),
        "max_depth": trial.suggest_int("max_depth", 1, 30),
        "num_leaves": trial.suggest_int("num_leaves", 2, 200),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 1.0),
        "subsample": trial.suggest_float("feature_fraction", 0.1, 1.0),
    }
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1121218)

    cv_scores = np.empty(10)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgb.LGBMClassifier(**param)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "multi_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [60]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, x, y)
study.optimize(func, n_trials=20)

[I 2022-12-22 13:31:12,139] A new study created in memory with name: LGBM Classifier


[1]	valid_0's multi_logloss: 1.46022
[2]	valid_0's multi_logloss: 1.11043
[3]	valid_0's multi_logloss: 0.889064
[4]	valid_0's multi_logloss: 0.719185
[5]	valid_0's multi_logloss: 0.5868
[6]	valid_0's multi_logloss: 0.488063
[7]	valid_0's multi_logloss: 0.406575
[8]	valid_0's multi_logloss: 0.458699
[9]	valid_0's multi_logloss: 0.385883
[10]	valid_0's multi_logloss: 0.331336
[11]	valid_0's multi_logloss: 0.282415
[12]	valid_0's multi_logloss: 0.302161
[13]	valid_0's multi_logloss: 0.262405
[14]	valid_0's multi_logloss: 0.225134
[15]	valid_0's multi_logloss: 0.194155
[16]	valid_0's multi_logloss: 0.168141
[17]	valid_0's multi_logloss: 0.146228
[18]	valid_0's multi_logloss: 0.127701
[19]	valid_0's multi_logloss: 0.113349
[20]	valid_0's multi_logloss: 0.100243
[21]	valid_0's multi_logloss: 0.10431
[22]	valid_0's multi_logloss: 0.0942893
[23]	valid_0's multi_logloss: 0.086539
[24]	valid_0's multi_logloss: 0.0786529
[25]	valid_0's multi_logloss: 0.0714527
[26]	valid_0's multi_logloss: 0.0654

[I 2022-12-22 13:31:35,773] Trial 0 finished with value: 0.040305419177923255 and parameters: {'learning_rate': 0.2033468548011935, 'max_depth': 26, 'num_leaves': 130, 'feature_fraction': 0.6690312927557149}. Best is trial 0 with value: 0.040305419177923255.


[1]	valid_0's multi_logloss: 1.83507
[2]	valid_0's multi_logloss: 1.72671
[3]	valid_0's multi_logloss: 1.63846
[4]	valid_0's multi_logloss: 1.55309
[5]	valid_0's multi_logloss: 1.47312
[6]	valid_0's multi_logloss: 1.40215
[7]	valid_0's multi_logloss: 1.33535
[8]	valid_0's multi_logloss: 1.36684
[9]	valid_0's multi_logloss: 1.30448
[10]	valid_0's multi_logloss: 1.24998
[11]	valid_0's multi_logloss: 1.19656
[12]	valid_0's multi_logloss: 1.21853
[13]	valid_0's multi_logloss: 1.17171
[14]	valid_0's multi_logloss: 1.1222
[15]	valid_0's multi_logloss: 1.07654
[16]	valid_0's multi_logloss: 1.033
[17]	valid_0's multi_logloss: 0.990623
[18]	valid_0's multi_logloss: 0.954472
[19]	valid_0's multi_logloss: 0.917978
[20]	valid_0's multi_logloss: 0.882305
[21]	valid_0's multi_logloss: 0.895108
[22]	valid_0's multi_logloss: 0.863948
[23]	valid_0's multi_logloss: 0.834546
[24]	valid_0's multi_logloss: 0.802785
[25]	valid_0's multi_logloss: 0.772654
[26]	valid_0's multi_logloss: 0.743804
[27]	valid_0's

[I 2022-12-22 13:31:58,690] Trial 1 finished with value: 0.43982222253750985 and parameters: {'learning_rate': 0.037842911018091334, 'max_depth': 29, 'num_leaves': 100, 'feature_fraction': 0.7443561444747373}. Best is trial 0 with value: 0.040305419177923255.


...

[I 2022-12-22 13:34:04,395] Trial 14 finished with value: 0.03007892477360081 and parameters: {'learning_rate': 0.2889935516075749, 'max_depth': 20, 'num_leaves': 80, 'feature_fraction': 0.5452517191418212}. Best is trial 5 with value: 0.02215267172075956.
[I 2022-12-22 13:34:04,470] Trial 15 pruned. Trial was pruned at iteration 1.


[1]	valid_0's multi_logloss: 1.42661
[1]	valid_0's multi_logloss: 1.09163
[2]	valid_0's multi_logloss: 0.676381
[3]	valid_0's multi_logloss: 0.437165
[4]	valid_0's multi_logloss: 0.313764
[5]	valid_0's multi_logloss: 0.214877
[6]	valid_0's multi_logloss: 0.168052
[7]	valid_0's multi_logloss: 0.12845
[8]	valid_0's multi_logloss: 0.151439
[9]	valid_0's multi_logloss: 0.119803
[10]	valid_0's multi_logloss: 0.100434
[11]	valid_0's multi_logloss: 0.0832396
[12]	valid_0's multi_logloss: 0.0892912
[13]	valid_0's multi_logloss: 0.0773967
[14]	valid_0's multi_logloss: 0.0636176
[15]	valid_0's multi_logloss: 0.0533213
[16]	valid_0's multi_logloss: 0.0474128
[17]	valid_0's multi_logloss: 0.0405834
[18]	valid_0's multi_logloss: 0.0362741
[19]	valid_0's multi_logloss: 0.0315788
[20]	valid_0's multi_logloss: 0.0294289
[21]	valid_0's multi_logloss: 0.0299453
[22]	valid_0's multi_logloss: 0.0275273
[23]	valid_0's multi_logloss: 0.0236234
[24]	valid_0's multi_logloss: 0.0219257
[25]	valid_0's multi_log

[I 2022-12-22 13:34:20,300] Trial 16 finished with value: 0.030396137230615912 and parameters: {'learning_rate': 0.46500246345408225, 'max_depth': 20, 'num_leaves': 125, 'feature_fraction': 0.5256398319357579}. Best is trial 5 with value: 0.02215267172075956.
[I 2022-12-22 13:34:20,351] Trial 17 pruned. Trial was pruned at iteration 0.
[I 2022-12-22 13:34:20,426] Trial 18 pruned. Trial was pruned at iteration 1.


[1]	valid_0's multi_logloss: 1.38678
[1]	valid_0's multi_logloss: 1.15516
[2]	valid_0's multi_logloss: 0.764725
[3]	valid_0's multi_logloss: 0.505541
[4]	valid_0's multi_logloss: 0.380366
[5]	valid_0's multi_logloss: 0.267044
[6]	valid_0's multi_logloss: 0.209094
[7]	valid_0's multi_logloss: 0.173784
[8]	valid_0's multi_logloss: 0.205081
[9]	valid_0's multi_logloss: 0.168247
[10]	valid_0's multi_logloss: 0.142225
[11]	valid_0's multi_logloss: 0.118293
[12]	valid_0's multi_logloss: 0.128016
[13]	valid_0's multi_logloss: 0.110434
[14]	valid_0's multi_logloss: 0.0919077
[15]	valid_0's multi_logloss: 0.0736377
[16]	valid_0's multi_logloss: 0.0623138
[17]	valid_0's multi_logloss: 0.0525485
[18]	valid_0's multi_logloss: 0.0453328
[19]	valid_0's multi_logloss: 0.0416178
[20]	valid_0's multi_logloss: 0.0355102
[21]	valid_0's multi_logloss: 0.0371803
[22]	valid_0's multi_logloss: 0.0331933
[23]	valid_0's multi_logloss: 0.0307869
[24]	valid_0's multi_logloss: 0.0267542
[25]	valid_0's multi_loglo

[I 2022-12-22 13:34:32,088] Trial 19 finished with value: 0.02926131064897062 and parameters: {'learning_rate': 0.42219355819621707, 'max_depth': 10, 'num_leaves': 29, 'feature_fraction': 0.45818201688930976}. Best is trial 5 with value: 0.02215267172075956.


In [61]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmse): 0.02215
	Best params:
		learning_rate: 0.3822953205658833
		max_depth: 12
		num_leaves: 39
		feature_fraction: 0.3911372880547346


In [62]:
from sklearn.model_selection import KFold, cross_val_score
model = lgb.LGBMClassifier(**study.best_params)
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(model, x, y, cv=skf)

In [63]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.99528302 0.99526066 0.99526066 0.99052133 1.         0.99526066
 0.97630332 1.         0.98104265 0.99526066]
Average CV Score:  0.9924192971474559
Number of CV Scores used in Average:  10
